In [81]:
import pandas as pd
from pathlib import Path

In [82]:
ruta_base = Path.cwd()
ruta_base = ruta_base.parents[1]

In [83]:
ruta_bd = ruta_base / 'Ejercicio2' / 'data' / 'processed' / 'clientes.csv'
clientes_raw = pd.read_csv(ruta_bd)

In [84]:
ruta_transacciones = ruta_base / 'Ejercicio2' / 'data' / 'processed' / 'transacciones.csv'
transacacciones = pd.read_csv(ruta_transacciones)

Opcion 1

In [85]:
#Esta función la realicé para eliminar aquellos registros de clientes que estuvieran duplicados con estado de la tarjeta 'inactiva'
# ya que no es posible determinar si las transacciones fueron cuando la tarjeta estaba activa, por lo cual no son objeto de estudio
# segun la suposición que estoy haciendo.

def filtrar_por_estado(grupo):
    # Si hay más de un registro para la misma IDENTIFICACIÓN (duplicado)
    if len(grupo) > 1:
        # Si todos los registros tienen el mismo estado
        if grupo['id_estado_tarjeta'].nunique() == 1:
            # Si el estado es 'Activa', se conserva el grupo completo
            if grupo['id_estado_tarjeta'].iloc[0] == 1:
                return grupo
            # Si el estado es 'Inactiva', se eliminan todos (devuelve un DataFrame vacío)
            else:
                return pd.DataFrame(columns=grupo.columns)
        else:
            # En caso de tener una mezcla (algunos 'Activa' y otros 'Inactiva'),
            # se conservan únicamente las filas activas.
            return grupo[grupo['id_estado_tarjeta'] == 1]
    else:
        # Si no hay duplicados, se retorna el grupo tal cual.
        return grupo

In [86]:
clientes_filtrados_raw = clientes_raw.groupby('IDENTIFICACIÓN', group_keys=False).apply(filtrar_por_estado)
clientes_filtrados_raw

C:\Users\Dalocu\AppData\Local\Temp\ipykernel_8748\1770378311.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  clientes_filtrados_raw = clientes_raw.groupby('IDENTIFICACIÓN', group_keys=False).apply(filtrar_por_estado)


,NOMBRE,IDENTIFICACIÓN,id_tipo_documento,id_clasificacion,id_tipo_tarjeta,id_estado_tarjeta,id_fecha
17,Santiago Vargas,2A3B4C5D6E7F8G9H0,3,1,1,1,11
0,Juan Pérez,2C3D4E5F6G7H8I9J0,1,1,1,1,132
12,Paula Mendoza,2C3D4E5F6G7H8I9J0,2,2,2,1,42
19,Sofía Sánchez,2E3F4G5H6I7J8K9L0,1,1,1,1,80
3,Ana Martínez,2G3H4I5J6K7L8M9N0,3,2,2,1,112
15,Fernando Castro,2G3H4I5J6K7L8M9N0,2,1,1,1,22
10,Elena Ruiz,2I3J4K5L6M7N8O9P0,1,2,2,2,60
6,Pedro González,2K3L4M5N6O7P8Q9R0,3,1,1,1,91
18,Natalia Herrera,2K3L4M5N6O7P8Q9R0,2,2,2,1,1
1,María García,2M3N4O5P6Q7R8S9T0,2,2,2,1,129


Opcion 2

In [87]:
# --- Función para contar el porcentaje de transacciones inválidas para cada cliente ---
def porcentaje_transacciones_invalidas(row, transacciones):
    """
    Para cada fila del DataFrame de clientes, filtra las transacciones que tengan el mismo valor en
    'IDENTIFICACIÓN' y calcula el porcentaje de aquellas cuyo id_fecha (fecha de la transacción) 
    es menor al id_fecha (fecha de apertura de la tarjeta).
    
    Retorna el porcentaje de transacciones inválidas. Si no hay transacciones asociadas,
    retorna 0.
    """
    ident = row['IDENTIFICACIÓN']
    apertura = row['id_fecha']  # Fecha de apertura de la tarjeta del cliente
    tx = transacciones[transacciones['IDENTIFICACION'] == ident]
    
    total_tx = len(tx)
    if total_tx == 0:
        return 0.0  # Sin transacciones asociadas, se retorna 0%
    
    invalidas = (tx['id_fecha'] < apertura).sum()
    porcentaje = (invalidas / total_tx) * 100 
    return porcentaje



In [88]:
# Aplicar la función a cada fila del DataFrame de clientes
clientes_raw['porc_transacciones_invalidas'] = clientes_raw.apply(
    lambda row: porcentaje_transacciones_invalidas(row, transacacciones), axis=1
)

In [89]:
clientes_raw

,NOMBRE,IDENTIFICACIÓN,id_tipo_documento,id_clasificacion,id_tipo_tarjeta,id_estado_tarjeta,id_fecha,porc_transacciones_invalidas
0,Juan Pérez,2C3D4E5F6G7H8I9J0,1,1,1,1,132,88.888889
1,María García,2M3N4O5P6Q7R8S9T0,2,2,2,1,129,100.000000
2,Carlos Rodríguez,2W3X4Y5Z6A7B8C9D0,1,1,1,2,118,85.714286
3,Ana Martínez,2G3H4I5J6K7L8M9N0,3,2,2,1,112,86.666667
4,Luis Hernández,2Q3R4S5T6U7V8W9X0,2,1,1,1,110,73.333333
5,Laura López,2A3B4C5D6E7F8G9H0,1,2,2,2,98,64.285714
6,Pedro González,2K3L4M5N6O7P8Q9R0,3,1,1,1,91,61.538462
7,Sofía Sánchez,2U3V4W5X6Y7Z8A9B0,1,2,2,1,84,66.666667
8,Marta Torres,2O3P4Q5R6S7T8U9V0,3,2,2,1,70,50.000000
9,Jorge Flores,2Y3Z4A5B6C7D8E9F0,2,1,1,1,63,81.818182


In [90]:
duplicados = clientes_raw[clientes_raw.duplicated(subset='IDENTIFICACIÓN', keep=False)]
duplicados = duplicados.sort_values('IDENTIFICACIÓN')
duplicados

,NOMBRE,IDENTIFICACIÓN,id_tipo_documento,id_clasificacion,id_tipo_tarjeta,id_estado_tarjeta,id_fecha,porc_transacciones_invalidas
5,Laura López,2A3B4C5D6E7F8G9H0,1,2,2,2,98,64.285714
17,Santiago Vargas,2A3B4C5D6E7F8G9H0,3,1,1,1,11,14.285714
0,Juan Pérez,2C3D4E5F6G7H8I9J0,1,1,1,1,132,88.888889
12,Paula Mendoza,2C3D4E5F6G7H8I9J0,2,2,2,1,42,66.666667
3,Ana Martínez,2G3H4I5J6K7L8M9N0,3,2,2,1,112,86.666667
15,Fernando Castro,2G3H4I5J6K7L8M9N0,2,1,1,1,22,6.666667
6,Pedro González,2K3L4M5N6O7P8Q9R0,3,1,1,1,91,61.538462
18,Natalia Herrera,2K3L4M5N6O7P8Q9R0,2,2,2,1,1,0.000000
1,María García,2M3N4O5P6Q7R8S9T0,2,2,2,1,129,100.000000
13,Fernando Castro,2M3N4O5P6Q7R8S9T0,1,1,1,2,36,10.000000


In [91]:
clientes_validos = clientes_raw[clientes_raw['porc_transacciones_invalidas'] == 0.0].copy()
clientes_validos = clientes_validos.drop(columns=['porc_transacciones_invalidas'])

clientes_validos

,NOMBRE,IDENTIFICACIÓN,id_tipo_documento,id_clasificacion,id_tipo_tarjeta,id_estado_tarjeta,id_fecha
18,Natalia Herrera,2K3L4M5N6O7P8Q9R0,2,2,2,1,1
19,Sofía Sánchez,2E3F4G5H6I7J8K9L0,1,1,1,1,80


In [92]:
carpeta_destino = ruta_base / 'Ejercicio2' / 'data' / 'processed'

clientes_validos.to_csv(carpeta_destino / 'clientes_validos.csv', index=False)